# Código 09: Cubo

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [ ]:
#Bibliotecas
!pip install glfw
import glfw
!pip install pyopengl
from OpenGL.GL import *
import OpenGL.GL.shaders #Não é redundante?
!pip install numpy
import numpy as np
import math
import random

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

#get_dim_pos: retorna tamanho da tela e posição da tela
def get_dim_pos(per_width = 0.6, per_height = 0.6): 
    # Obtendo configurações do monitor
    monitores = glfw.get_monitors()
    monitor = monitores[0]
    video_mode = glfw.get_video_mode(monitor)
    WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
    # Definindo proporção que se quer do monitor
    WIDTH_WINDOW : int = int(per_width*WIDTH_WINDOW)
    HEIGHT_WINDOW : int = int(per_height*HEIGHT_WINDOW)
    POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
    POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
    return WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW

# Pega tamanho da tela e posição da tela
WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW = get_dim_pos(0.6,0.6)
# Criando janela
TITLE: str = "Cubo"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Eventos de teclado e mouse

In [ ]:
angle = 0.0
scale = 1.0
pos_x = pos_y = pos_z = 0.0

def key_event(window,key,scancode,action,mods):
    global angle, pos_x, pos_y, scale

    angular_speed = 0.139
    if key == 81: angle -= angular_speed #Tecla Q
    if key == 82: angle += angular_speed #Tecla R

    scale_speed = 0.2
    if key == 70: scale += scale_speed #Tecla F
    if key == 71: scale -= scale_speed #Tecla G

    linear_speed = 0.05
    if key == 87: pos_y += linear_speed #Tecla W
    if key == 83: pos_y -= linear_speed #Tecla S
    if key == 65: pos_x -= linear_speed #Tecla A
    if key == 68: pos_x += linear_speed #Tecla D

    # print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)
    # print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    
glfw.set_mouse_button_callback(window,mouse_event)

### Shaders: Vertex e Fragment

Aqui, como será trabalhado em terceira dimensão, cria-se um atributo `position` com três dimensões que será convertido para quatro pela coordenada homogênea. Tem-se também `mat_transformation` com quatro dimensões. 

In [ ]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Solicitando espaço, compilando e linkando

In [ ]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Criação dos vértices

Para este exemplo, usaremos 24 vértices. Porém, o cubo pode utilizar apenas quatro. A ideia aqui é aplicar transformações a cada uma das seis faces. Outro ponto importante é que cada ponto possui três coordenadas.

In [ ]:
#Criando espaço
vertices = np.zeros(24, [("position", np.float32, 3)])

#Preenchendo as coordenadas
vertices['position'] = [
    # Face 1 do Cubo (vértices do quadrado)
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),

    # Face 2 do Cubo
    (+0.2, -0.2, +0.2),
    (+0.2, -0.2, -0.2),         
    (+0.2, +0.2, +0.2),
    (+0.2, +0.2, -0.2),
    
    # Face 3 do Cubo
    (+0.2, -0.2, -0.2),
    (-0.2, -0.2, -0.2),            
    (+0.2, +0.2, -0.2),
    (-0.2, +0.2, -0.2),

    # Face 4 do Cubo
    (-0.2, -0.2, -0.2),
    (-0.2, -0.2, +0.2),         
    (-0.2, +0.2, -0.2),
    (-0.2, +0.2, +0.2),

    # Face 5 do Cubo
    (-0.2, -0.2, -0.2),
    (+0.2, -0.2, -0.2),         
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    
    # Face 6 do Cubo
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),           
    (-0.2, +0.2, -0.2),
    (+0.2, +0.2, -0.2)
]

### Manipulação dos espaços de dados

In [ ]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando. Também captura a posicao de loc_color e loc_mat
loc_position = glGetAttribLocation(program, "position")
loc_color = glGetUniformLocation(program, "color")
loc_mat = glGetUniformLocation(program, "mat_transformation")
glEnableVertexAttribArray(loc_position)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_position, 3, GL_FLOAT, False, stride, offset)

### Exibindo na tela

In [ ]:
glfw.show_window(window)

## Loop principal

* `glEnable()`: ativa recursos específicos do OpenGL. Quando se usa `GL_DEPTH_TEST` como parâmetro, habilita-se o teste de profundidade. Este teste determina qual objeto ou fragmento de cena deve ser visível em um determinado pixel na tela comparando a profundidade (ou distância) com relação à câmera. Caso o fragmento que estiver no buffer for mais próximo que aquele em análise, ele é jogado fora. Caso contrário, ele substitui o fragmento que estiver no buffer.

In [ ]:
# Variavel angulo usado para visualizar
glEnable(GL_DEPTH_TEST) ### importante para 3D

def calc_mat(a,b,c):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = c.reshape(4,4)
    return np.dot(np.dot(m_a, m_b), m_c)

while not glfw.window_should_close(window):

    glfw.poll_events()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)

    mat_translation = np.array([    1.0, 0.0, 0.0, pos_x, 
                                    0.0, 1.0, 0.0, pos_y, 
                                    0.0, 0.0, 1.0, pos_z, 
                                    0.0, 0.0, 0.0,   1.0], np.float32)
    
    scale_x = scale_y = scale_z = scale
    mat_scale =     np.array([      scale_x,     0.0,     0.0, 0.0, 
                                        0.0, scale_y,     0.0, 0.0, 
                                        0.0,     0.0, scale_z, 0.0, 
                                        0.0,     0.0,     0.0, 1.0], np.float32)
    
    # Apenas para visualizar o cubo rotacionando
    # Modifica o angulo de rotacao em cada iteracao
    cos = math.cos(angle)
    sin = math.sin(angle)
    mat_rotation_z = np.array([     cos, -sin, 0.0, 0.0, 
                                    sin,  cos, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos, -sin, 0.0, 
                                    0.0, sin,  cos, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos,  0.0, sin, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin, 0.0, cos, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_transform = calc_mat(mat_rotation_x, mat_rotation_z, mat_rotation_y)
    mat_transform = calc_mat(mat_translation, mat_scale, mat_transform)
    glUniformMatrix4fv(loc_mat, 1, GL_TRUE, mat_transform)

    glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
   
    glUniform4f(loc_color, 1, 0, 0, 1.0) ### vermelho
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)
    
    glUniform4f(loc_color, 0, 0, 1, 1.0) ### azul
    glDrawArrays(GL_TRIANGLE_STRIP, 4, 4)
    
    glUniform4f(loc_color, 0, 1, 0, 1.0) ### verde
    glDrawArrays(GL_TRIANGLE_STRIP, 8, 4)
    
    glUniform4f(loc_color, 1, 1, 0, 1.0) ### amarela
    glDrawArrays(GL_TRIANGLE_STRIP, 12, 4)
    
    glUniform4f(loc_color, 0.5, 0.5, 0.5, 1.0) ### cinza
    glDrawArrays(GL_TRIANGLE_STRIP, 16, 4)
    
    glUniform4f(loc_color, 0.5, 0, 0, 1.0) ### marrom
    glDrawArrays(GL_TRIANGLE_STRIP, 20, 4)
    
    glfw.swap_buffers(window)

glfw.terminate()